In [13]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [11]:
# !pip install fugashi
# !pip install ipadic

In [14]:
# Load pre-trained tokenizer
# tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
import pickle

with open('data_extract/ontology.pickle', 'rb') as f:
    ontology = pickle.load(f)

## grand parent taxon

In [15]:
ontology_d = {}
for d in ontology:
    ontology_d[d["id"]]=d

In [17]:
for d in ontology:
    try:
        d["grand_parent_taxon"] = ontology_d[d["id"]]["parent_taxon"]
    except:
        pass

In [18]:
grand_
ontology[0:10]

[{'id': 'Q27638142',
  'en_name': 'Diglossa albilatera albilatera',
  'ja_name': None,
  'en_aliases': {},
  'ja_aliases': {},
  'img_urls': {},
  'taxon_rank': 'Q68947',
  'taxon_name': 'Diglossa albilatera albilatera',
  'parent_taxon': 'Q536912',
  'grand_parent_taxon': 'Q536912'},
 {'id': 'Q40506530',
  'en_name': 'Lochmias nematura castanonota',
  'ja_name': None,
  'en_aliases': {},
  'ja_aliases': {},
  'img_urls': {},
  'taxon_rank': 'Q68947',
  'taxon_name': 'Lochmias nematura castanonota',
  'parent_taxon': 'Q1039940',
  'grand_parent_taxon': 'Q1039940'},
 {'id': 'Q27608560',
  'en_name': 'Gallinago nigripennis nigripennis',
  'ja_name': None,
  'en_aliases': {},
  'ja_aliases': {},
  'img_urls': {},
  'taxon_rank': 'Q68947',
  'taxon_name': 'Gallinago nigripennis nigripennis',
  'parent_taxon': 'Q1260623',
  'grand_parent_taxon': 'Q1260623'},
 {'id': 'Q15977273',
  'en_name': 'Phragmacia',
  'ja_name': None,
  'en_aliases': {},
  'ja_aliases': {},
  'img_urls': {'0': 'https:

In [ ]:
# Tokenize input
text = 'テレビでサッカーの試合を見る。'
tokenized_text = tokenizer.tokenize(text)
# ['テレビ', 'で', 'サッカー', 'の', '試合', 'を', '見る', '。']

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 2
tokenized_text[masked_index] = '[MASK]'
# ['テレビ', 'で', '[MASK]', 'の', '試合', 'を', '見る', '。']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# [571, 12, 4, 5, 608, 11, 2867, 8]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
# tensor([[ 571,   12,    4,    5,  608,   11, 2867,    8]])

# Load pre-trained model
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model.eval()

# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0][0, masked_index].topk(5) # 予測結果の上位5件を抽出

# Show results
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()
    token = tokenizer.convert_ids_to_tokens([index])[0]
    print(i, token)